# Simulação de aplicação

O paradigma SSVEP é bastante utilizado em aplicações reais por ser fácil de evocar os sinais e posteriormente classificá-los.
    
Para processar uma aplicação SSVEP em tempo real é necessário calcular a máxima energia dos _targets_ em janelas ou _buffers_ suficientemente grandes. O tamanho da janela de dados é inversamente proporcional a taxa de amostragem do equipamente de EEG utilizado. Ou seja, quanto maior a taxa de amostragem, menor o tempo decessário para calcular a **energia deslizante do sinal**
    
**Tarefa**: simule uma aplicação SSVEP criando um buffer do sinal no domínio do tempo em uma única dimensão. Se inicialmente a dimensão dos dados era `(100, 8192)`, agora será `(819200)`. Crie um buffer de 3 ou 4 segundos, informando a cada segundo qual está sendo a frequência evocada.

In [20]:
import numpy as np
import mne
from scipy.signal import welch
import json

# Carregar os dados no domínio do tempo
epochs = mne.read_epochs('ssvep-epo.fif')
data = epochs.get_data().squeeze().flatten()  # shape de (100, 1, 8192) -> (100, 8192) -> (819200,)

# Parâmetros do buffer
sampling_rate = 512  # Taxa de amostragem do EEG (Hz)
buffer_size = 4  # Tamanho do buffer em segundos
step_size = 1  # Tamanho do passo do buffer em segundos

buffer_samples = buffer_size * sampling_rate  # Tamanho do buffer em amostras
step_samples = step_size * sampling_rate  # Passo do buffer em amostras

# Frequências esperadas (para simulação)
frequencies = [6.0, 6.5, 7.0, 7.5, 8.2, 9.3]

# Resultados finais para exportação
results = []

# Janela para média móvel (em buffers)
moving_average_window = 3
recent_predictions = []

# Simulação do buffer
print("Simulação de aplicação SSVEP:")
for start in range(0, len(data) - buffer_samples, step_samples):  # Avançar 1 segundo
    # Extrair o segmento do buffer
    buffer = data[start : start + buffer_samples]
    
    # Calcular a PSD do buffer usando Welch
    freqs, psd = welch(buffer, fs=sampling_rate, nperseg=buffer_samples)
    
    # Encontrar a frequência com maior energia
    max_idx = np.argmax(psd)
    predicted_freq = freqs[max_idx]
    
    # Comparar com as frequências esperadas usando tolerância de ±0.5 Hz
    closest_freq = min(frequencies, key=lambda x: abs(x - predicted_freq))
    within_tolerance = abs(predicted_freq - closest_freq) <= 0.5
    
    # Adicionar à janela de média móvel
    recent_predictions.append(closest_freq if within_tolerance else None)
    if len(recent_predictions) > moving_average_window:
        recent_predictions.pop(0)
    
    # Calcular a frequência dominante (média móvel simples)
    valid_predictions = [p for p in recent_predictions if p is not None]
    dominant_freq = np.mean(valid_predictions) if valid_predictions else None
    
    # Exibir os resultados
    print(f"Buffer {start // step_samples + 1}:")
    print(f"  Frequência evocada estimada: {predicted_freq:.2f} Hz")
    print(f"  Frequência mais próxima dos alvos: {closest_freq:.2f} Hz")
    print(f"  Frequência dominante (média móvel): {dominant_freq:.2f} Hz\n" if dominant_freq else "  Sem frequência dominante detectada\n")
    
    # Armazenar o resultado
    results.append({
        "buffer_number": start // step_samples + 1,
        "predicted_frequency": predicted_freq,
        "closest_target_frequency": closest_freq,
        "within_tolerance": within_tolerance,
        "dominant_frequency": dominant_freq
    })

# Exportar os resultados para um arquivo JSON
with open("ssvep_results.json", "w") as f:
    json.dump(results, f, indent=4, default=str)


Reading c:\Users\catar\Downloads\multi\multi\src\ssvep-epo.fif ...
Isotrak not found
    Found the data of interest:
        t =       0.00 ...   15998.05 ms
        0 CTF compensation matrices available
Not setting metadata
100 matching events found
No baseline correction applied
0 projection items activated
Simulação de aplicação SSVEP:
Buffer 1:
  Frequência evocada estimada: 7.50 Hz
  Frequência mais próxima dos alvos: 7.50 Hz
  Frequência dominante (média móvel): 7.50 Hz

Buffer 2:
  Frequência evocada estimada: 7.50 Hz
  Frequência mais próxima dos alvos: 7.50 Hz
  Frequência dominante (média móvel): 7.50 Hz

Buffer 3:
  Frequência evocada estimada: 7.50 Hz
  Frequência mais próxima dos alvos: 7.50 Hz
  Frequência dominante (média móvel): 7.50 Hz

Buffer 4:
  Frequência evocada estimada: 7.75 Hz
  Frequência mais próxima dos alvos: 7.50 Hz
  Frequência dominante (média móvel): 7.50 Hz

Buffer 5:
  Frequência evocada estimada: 7.50 Hz
  Frequência mais próxima dos alvos: 7.50 Hz
 